In [11]:
import requests
from bs4 import BeautifulSoup
import csv
import re

# Tarayicimizin user agent ini veriyoruz
headers = {"User-Agent" : "Mozilla/5.0 (X11; Linux x86_64; rv:107.0) Gecko/20100101 Firefox/107.0"}
url = 'https://www.resmigazete.gov.tr'

# csv dosyasini aciyoruz. Eger onceden bu isimde bir csv dosyasi var ise uzerine yazacaktir.
csvfile = open('resmigazete.csv', 'w', newline='', encoding='utf-8')
csvwriter = csv.writer(csvfile)
# csv dosyasina sutun basliklarini ekliyoruz
sutunlar = ["tarih", "sayi", "baslik", "altbaslik", "fihrist", "link"]
csvwriter.writerow(sutunlar)

kac_gun = input("Bugün dahil geriye doğru kaç günlük resmi gazete sayfasını scraping yapmak istiyorsunuz? Lütfen gün sayısı giriniz:")
i = 1
while i <= int(kac_gun):
    i += 1
    r = requests.get(url, headers=headers)
    if r.status_code == 200:
        print("Sayfa başarıyla yüklendi")
        # Sayfadan gelen veriyi html sonra da soup nesnesine donusturuyoruz
        html = r.text
        soup = BeautifulSoup(html, 'html.parser')
        
        # div lerin bulundugu html-content class olan ana div i buluyoruz
        html_content = soup.find('div', class_='html-content')
        
        # bir sonraki sayfanin linkini hazirliyoruz
        url = 'https://www.resmigazete.gov.tr' + soup.find(id='oncekiSonraki').a['href']
        print("Bir sonraki link: " + url)
        
        # sayfadan tarih ve sayiyi cekiyoruz
        tarih_ve_sayi = soup.find('span', id='spanGazeteTarih')
        if tarih_ve_sayi != None:
            text = tarih_ve_sayi.string.split()
            tarih = text[0]
            # Python3.10 istiyor bu match case statement
            match text[1]:
                case "Ocak":
                    tarih += "/01/"
                case "Şubat":
                    tarih += "/02/"
                case "Mart":
                    tarih += "/03/"
                case "Nisan":
                    tarih += "/04/"
                case "Mayıs":
                    tarih += "/05/"
                case "Haziran":
                    tarih += "/06/"
                case "Temmuz":
                    tarih += "/07/"
                case "Ağustos":
                    tarih += "/08/"
                case "Eylül":
                    tarih += "/09/"
                case "Ekim":
                    tarih += "/10/"
                case "Kasım":
                    tarih += "/11/"
                case "Aralık":
                    tarih += "/12/"
            tarih += text[2]
            print("Bugün'ün tarihi: " + tarih)
            sayi = text[5]
            print(sayi)
        
            
        
        #DEGISKENLER#        
        baslik = "YOK"
        altbaslik = "YOK"
        fihrist = "YOK"
        fihrist_link = "YOK"
        
        divs = html_content.find_all('div')        
        for div in divs:
            if div['class'][0] == "card-title":
                baslik = div.string
                altbaslik = "YOK"
                #print(baslik)
            elif div['class'][0] == "html-subtitle":
                altbaslik = div.string
                #print(altbaslik)
            elif div['class'][0] == "fihrist-item":
                fihrist = div.string
                if div.a:
                    fihrist_link = div.a['href']
                    x = re.findall("^https://www.resmigazete.gov.tr/", fihrist_link)
                    if not x:
                        fihrist_link = "https://www.resmigazete.gov.tr" + fihrist_link

                #print(tarih+" "+baslik+" "+altbaslik+" "+fihrist+" "+fihrist_link)
                fihrist_bilgileri = [tarih, sayi, baslik, altbaslik, fihrist, fihrist_link]
                csvwriter.writerow(fihrist_bilgileri)
csvfile.close()

Bugün dahil geriye doğru kaç günlük resmi gazete sayfasını scraping yapmak istiyorsunuz? Lütfen gün sayısı giriniz: 3


Sayfa başarıyla yüklendi
Bir sonraki link: https://www.resmigazete.gov.tr/06.01.2023
Bugün'ün tarihi: 07/01/2023
32066
Sayfa başarıyla yüklendi
Bir sonraki link: https://www.resmigazete.gov.tr/05.01.2023
Bugün'ün tarihi: 06/01/2023
32065
Sayfa başarıyla yüklendi
Bir sonraki link: https://www.resmigazete.gov.tr/04.01.2023
Bugün'ün tarihi: 05/01/2023
32064
